In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import subprocess
from hscpy import realisation, parameters
from hscpy.figures import burden as burden_figures
from hscpy.figures import sfs as sfs_figures
from hscpy.figures import PlotOptions, simulations

from futils import parse_version

SAVEFIG = True
BIGLABELS = False
FIGSIZE = [5, 3] if BIGLABELS else [6.4, 4.8]  # default matplotlib
EXTENSION = ".svg"
PLOT_OPTIONS = PlotOptions(figsize=FIGSIZE, extension=EXTENSION, save=SAVEFIG)
PATH2HSC = Path("~").expanduser() / "hsc"
assert PATH2HSC.is_dir()

In [ ]:
%%bash -s "$PATH2HSC" --out version
$1/target/release/hsc  --version

In [ ]:
VERSION = parse_version(version)
print("Running hsc with version:", VERSION)

In [ ]:
# PARAMS
tau = 4
mu = 5
c = 100_000 # cells
# fitness distr (measured per year)
eta, sigma = 0.06, 0.02

cmd = f"{PATH2HSC}/target/release/hsc -c {c} -y 81 -r 1 --tau {tau} --mu0 {mu} --mu-exp {parameters.compute_m_exp(c)} --mu-division 1.2 --mu-background {parameters.compute_m_background(tau)} --mean-std {parameters.compute_s_per_division_from_s_per_year(eta, tau)} {parameters.compute_std_per_division_from_std_per_year(sigma, tau)} --subsamples 368 --snapshots 0 10 20 30 40 50 60 70 80 --save-population --sequential ./simulations"
print(cmd)

In [ ]:
subprocess.run(
    cmd,
    shell=True, 
    check=True
)

In [ ]:
sfs = realisation.load_realisations(
    Path("./simulations/100000cells/sfs/"),
    realisation.RealisationKind.SFS,
    filtering=None
)

In [ ]:
sfs[0][0].sfs[53744]

In [ ]:
max(sfs[0][0].sfs.keys()) * 2 

In [ ]:
for k, sfs_ in sfs.items():
    fig, ax = plt.subplots(1, 1, figsize=[7.8, 6], layout="constrained") 
    sfs_figures.plot_sfs(
        ax,
        sfs_[0].sfs, 
        True, 
        PLOT_OPTIONS, 
        normalise_x=sfs_figures.ToCellFrequency(sample_size=100_000, to_one=True),
        ls="",
        marker="o",
        mew=2,
        label=f"{int(k)} years"
    )
    ax.set_xlim([10**(-5), 3])
    ax.legend()
    plt.savefig(f"{int(k)}_sfs.png")
    plt.show()

### No exp. phase

In [ ]:
# PARAMS
tau = 4
mu = 5
c = 100_000 # cells
# fitness distr (measured per year)
eta, sigma = 0.06, 0.02

cmd = f"{PATH2HSC}/target/release/hsc -c {c} -y 81 -r 1 --tau {tau} --mu0 {mu} --mu-division 1.2 --mu-background {parameters.compute_m_background(tau)} --mean-std {parameters.compute_s_per_division_from_s_per_year(eta, tau)} {parameters.compute_std_per_division_from_std_per_year(sigma, tau)} --subsamples 368 --snapshots 0 10 20 30 40 50 60 70 80 --save-population --sequential ./simulations_exp"
print(cmd)

In [ ]:
subprocess.run(
    cmd,
    shell=True, 
    check=True
)

In [ ]:
sfs = realisation.load_realisations(
    Path("./simulations_exp/368cells/sfs/"),
    realisation.RealisationKind.SFS,
    filtering=None
)

In [ ]:
for k, sfs_ in sfs.items():
    fig, ax = plt.subplots(1, 1, figsize=[7.8, 6], layout="constrained") 
    sfs_figures.plot_sfs(
        ax,
        sfs_[0].sfs, 
        True, 
        PLOT_OPTIONS, 
        normalise_x=sfs_figures.ToCellFrequency(sample_size=100_000, to_one=True),
        ls="",
        marker="o",
        mew=2,
        label=f"{int(k)} years"
    )
    ax.set_xlim([10**(-5), 3])
    ax.legend()
    plt.savefig(f"{int(k)}_sfs.png")
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, layout="constrained")
ax.bar([ele / 4 for ele in [1, 2, 3, 4]], [0.5, 0.333, 0, 0.167], width=0.2)
ax.set_xlabel("density of cells")
ax.set_ylabel("density of variants")
ax.set_xticks([0.25, 0.5, 0.75, 1])
plt.savefig("vaf.png")